<a href="https://colab.research.google.com/github/jamilly-b/curso-machine-learning/blob/main/Projeto/letterboxd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np

In [2]:
drive.mount('/content/drive')
dados_dos_filmes = pd.read_csv('/content/drive/MyDrive/machine-learning-project/movie_data.csv', lineterminator='\n')
usuario = pd.read_csv('/content/drive/MyDrive/machine-learning-project/users_export.csv')
avaliacao = pd.read_csv('/content/drive/MyDrive/machine-learning-project/ratings_export.csv')

Mounted at /content/drive


## Visualização dos dados

### Tabela Usuário

In [3]:
usuario.head() 

,_id,display_name,num_ratings_pages,num_reviews,username
0,5fc4172ec6cd28ebd99dd0e2,Lucy,32.0,1650,deathproof
1,5fc4172ec6cd28ebd99dd0ea,Matt Singer,52.0,1915,superpulse
2,5fc4172ec6cd28ebd99dd0ed,Sean Baker,21.0,1283,lilfilm
3,5fc4172ec6cd28ebd99dd0ee,iana,37.0,1177,ianamurray
4,5fc419171ebf67b9fbe48615,Lizzy,57.0,1810,punchdrunklizzy


In [4]:
usuario.columns

Index(['_id', 'display_name', 'num_ratings_pages', 'num_reviews', 'username'], dtype='object')

**_id** = ID exclusivo global.

**display_name** = Nome de exibição em Letterboxd no momento do último rastreamento. ###################

**num_ratings_pages** = Número de páginas de avaliações. ###################

**num_reviews** = Número total de avaliações feitas por este usuário.

**username** = Letterboxd ID/nome de usuário exclusivo. Pode ser usado para formar um URL Letterboxd.

In [5]:
usuario.isnull().sum()

_id                     0
display_name          307
num_ratings_pages    3392
num_reviews             0
username                0
dtype: int64

In [6]:
usuario.index.value_counts().sum()

8139

### Tabela Dados dos filmes

In [7]:
dados_dos_filmes.head(3)

,_id,genres,image_url,imdb_id,imdb_link,movie_id,movie_title,original_language,overview,popularity,production_countries,release_date,runtime,spoken_languages,tmdb_id,tmdb_link,vote_average,vote_count,year_released
0,5fc85f606758f69634496fd3,"[""Music"",""Animation""]",film-poster/4/6/4/4/4/0/464440-football-freaks...,NaN,NaN,football-freaks,Football Freaks,en,"Football crazy, football mad. Don’t watch this...",0.600,"[""United Kingdom""]",1971-12-05,0.0,[],535272.0,https://www.themoviedb.org/movie/535272/,0.0,0.0,1971.0
1,5fc85ff26758f696344ace0c,[],film-poster/2/4/5/5/0/0/245500-aftermath-0-230...,tt0586129,http://www.imdb.com/title/tt0586129/maindetails,aftermath-1960,Aftermath,en,Aftermath was the pilot for an unsold TV serie...,0.600,[],1960-04-17,22.0,[],318331.0,https://www.themoviedb.org/movie/318331/,8.0,1.0,1960.0
2,5fc85f606758f69634496fcd,"[""Drama""]",film-poster/9/3/3/1/8/93318-where-chimneys-are...,tt0045731,http://www.imdb.com/title/tt0045731/maindetails,where-chimneys-are-seen,Where Chimneys Are Seen,ja,Gosho’s most celebrated film both in Japan and...,1.568,"[""Japan""]",1953-03-05,108.0,"[""日本語""]",117779.0,https://www.themoviedb.org/movie/117779/,6.6,10.0,1953.0


In [8]:
dados_dos_filmes.columns

Index(['_id', 'genres', 'image_url', 'imdb_id', 'imdb_link', 'movie_id',
       'movie_title', 'original_language', 'overview', 'popularity',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'tmdb_id', 'tmdb_link', 'vote_average', 'vote_count', 'year_released'],
      dtype='object')

**_id** = ID exclusivo global.

**genres** = Array  de gêneros de filmes associados.

**image_url** = Imagem usada no site Letterboxd para o filme. Geralmente um pôster. ##############

**imdb_id** = ID para IMDB. 

**imdb_link** = Link para a página IMDB do filme. ##############

**movie_id** = Identificação do filme Letterboxd. 

**movie_title** = Título de exibição. 

**original_language** = Idioma original do filme 

**overview** = Descrição do filme ##############

**popularity** = Popularidade numérica  

**production_countries** = Matriz de países em que o filme foi produzido 

**release_date** = Data de lançamento ##############

**runtime** = Tempo de execução em minutos 

**spoken_languages** = Array de todos os idiomas falados no filme ##############

**tmdb_id** = ID para acessar dados na API 

**tmdb_link** = Link para a entrada do filme no TMDB 

**vote_average** = Média de votos 

**vote_count** = contagem de votos 

**year_released** = Ano lançado

In [9]:
dados_dos_filmes.shape

(285504, 19)

In [10]:
dados_dos_filmes.isna().sum()

_id                         0
genres                  10632
image_url               21097
imdb_id                 41702
imdb_link               41702
movie_id                    2
movie_title              2164
original_language       10632
overview                24256
popularity              10632
production_countries    10632
release_date            14454
runtime                 15449
spoken_languages        10632
tmdb_id                  5587
tmdb_link                5587
vote_average            10632
vote_count              10632
year_released            7572
dtype: int64

### Tabela Avaliação

In [11]:
avaliacao.head()

,_id,movie_id,rating_val,user_id
0,5fc57c5d6758f6963451a07f,feast-2014,7,deathproof
1,5fc57c5d6758f6963451a063,loving-2016,7,deathproof
2,5fc57c5d6758f6963451a0ef,scripted-content,7,deathproof
3,5fc57c5d6758f6963451a060,the-future,4,deathproof
4,5fc57c5c6758f69634519398,mank,5,deathproof


In [12]:
avaliacao.shape

(11078167, 4)

In [13]:
avaliacao.columns

Index(['_id', 'movie_id', 'rating_val', 'user_id'], dtype='object')

**_id** = ID exclusivo global.

**movie_id** = Letterboxd ID exclusivo para o filme. Pode ser usado para corresponder aos dados do filme neste conjunto de dados ou para construir um Letterboxd

**rating_val** = Valor da classificação com base nas estrelas dadas (em incrementos de meia estrela). Uma meia estrela é 1, cinco estrelas é 10, etc.

**user_id** = Letterboxd ID exclusivo para o usuário. Pode ser usado para corresponder aos dados do usuário neste conjunto de dados ou para construir um Letterboxd

In [14]:
avaliacao.isnull().sum()

_id           0
movie_id      6
rating_val    0
user_id       0
dtype: int64

## Tratamento dos dados

plotar graficos sobre: número médio de reviews, generos, tempo medio dos filmes, pais de produção.

############### Antes dessa parte, cuidar dos dados nulos e das listas vazias

In [15]:
avaliacao = avaliacao.rename(columns={'user_id': 'username'})

In [16]:
tabela = pd.merge(usuario, avaliacao, on = 'username', how = 'inner')
tabela = pd.merge(tabela, dados_dos_filmes, on = 'movie_id', how = 'inner')
tabela.head()

,_id_x,display_name,num_ratings_pages,num_reviews,username,_id_y,movie_id,rating_val,_id,genres,...,popularity,production_countries,release_date,runtime,spoken_languages,tmdb_id,tmdb_link,vote_average,vote_count,year_released
0,5fc4172ec6cd28ebd99dd0e2,Lucy,32.0,1650,deathproof,5fc57c5d6758f6963451a07f,feast-2014,7,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
1,5fc419171ebf67b9fbe485a2,Allison M. 🌱 @ SXSW,290.0,3105,allisoncm,5fc67eda6758f69634d57dba,feast-2014,4,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
2,5fc4172ec6cd28ebd99dd0f1,{Todd},78.0,5271,juggernaut323,5fc67dae6758f69634cd2410,feast-2014,9,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
3,5fc4172ec6cd28ebd99dd0ec,DirkH,65.0,2029,dirkh,5fc57cbd6758f696345475a3,feast-2014,8,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
4,5fc419161ebf67b9fbe48571,lauren,24.0,1117,nycsubwayrat,5fc57d246758f696345689a5,feast-2014,10,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0


In [17]:
tabela.columns.to_list()

['_id_x',
 'display_name',
 'num_ratings_pages',
 'num_reviews',
 'username',
 '_id_y',
 'movie_id',
 'rating_val',
 '_id',
 'genres',
 'image_url',
 'imdb_id',
 'imdb_link',
 'movie_title',
 'original_language',
 'overview',
 'popularity',
 'production_countries',
 'release_date',
 'runtime',
 'spoken_languages',
 'tmdb_id',
 'tmdb_link',
 'vote_average',
 'vote_count',
 'year_released']

In [18]:
tabela = tabela[['_id_x','num_reviews','username','_id_y','movie_id','rating_val','_id','genres','image_url','imdb_id','movie_title','original_language','popularity','production_countries','runtime','tmdb_id','tmdb_link','vote_average','vote_count','year_released']]

In [19]:
tabela.isna().sum()

_id_x                        0
num_reviews                  0
username                     0
_id_y                        0
movie_id                    12
rating_val                   0
_id                          0
genres                  167900
image_url                39956
imdb_id                 115587
movie_title               9485
original_language       167900
popularity              167900
production_countries    167900
runtime                 174705
tmdb_id                  93552
tmdb_link                93552
vote_average            167900
vote_count              167900
year_released            20246
dtype: int64

In [20]:
tabela = tabela.dropna(subset= ['movie_id'])

inserindo cada tributo da tabela resultante em um vetor para melhor manipulação dos dados

In [21]:
df = pd.DataFrame()

In [22]:
vetor__id_x = np.array(tabela._id_x)
vetor_num_reviews = np.array(tabela.num_reviews)
vetor_username = np.array(tabela.username)
vetor__id_y = np.array(tabela._id_y)
vetor_movie_id = np.array(tabela.movie_id)
vetor_rating_val = np.array(tabela.rating_val)
vetor__id = np.array(tabela._id)
vetor_genres = np.array(tabela.genres)
vetor_image_url = np.array(tabela.image_url)
vetor_imdb_id = np.array(tabela.imdb_id)
vetor_movie_title = np.array(tabela.movie_title)
vetor_original_language = np.array(tabela.original_language)
vetor_popularity = np.array(tabela.popularity)
vetor_production_countries = np.array(tabela.production_countries)
vetor_runtime = np.array(tabela.runtime)
vetor_tmdb_id = np.array(tabela.tmdb_id)
vetor_tmdb_link = np.array(tabela.tmdb_link)
vetor_vote_average = np.array(tabela.vote_average)
vetor_vote_count = np.array(tabela.vote_count)
vetor_year_released = np.array(tabela.year_released)

In [23]:
Colunas =['ID_usuario','Num_reviews','Username','ID_Avaliacao','Codigo_Filme','Nota_usuario','ID_Filme','Generos','Link_Poster','IMDB_ID','Filme','Idioma_Original','Popularidade','Pais_Producao','Minutos_Filme','TMDB_ID','Link_TMDB','Nota_Media','Contagem_TMDB','Ano_Lancamento']

In [24]:
df = pd.DataFrame(columns = Colunas)
df.ID_usuario = vetor__id_x
df.Username = vetor_username
df.ID_Avaliacao = vetor__id_y
df.Codigo_Filme = vetor_movie_id
df.Nota_usuario = vetor_rating_val
df.ID_Filme = vetor__id
df.Generos = vetor_genres
df.Filme = vetor_movie_title
df.Popularidade = vetor_popularity
df.Pais_Producao = vetor_production_countries
df.Minutos_Filme = vetor_runtime
df.Nota_Media = vetor_vote_average
df.Contagem_TMDB = vetor_vote_count
df.Ano_Lancamento = vetor_year_released

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-0fd044c60909>", line 7, in <module>
    df.ID_Filme = vetor__id
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py", line 5516, in __setattr__
    self[name] = value
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py", line 3612, in __setitem__
    self._set_item(key, value)
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py", line 3784, in _set_item
    value = self._sanitize_column(value)
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py", line 4510, in _sanitize_column
    return sanitize_array(value, self.index, copy=True, allow_2d=True)
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/construction.py", line 545, in sanitize_array
    subarr = _try_cast(data, dtype, copy, raise_cast_fai

KeyboardInterrupt: ignored

In [ ]:
df.info()

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

Removendo os elemento não numéricos:

In [ ]:
df['ID_usuario'] = df['ID_usuario'].str.replace(r'\D', '')
df['ID_Avaliacao'] = df['ID_Avaliacao'].str.replace(r'\D', '')
df['ID_Filme'] = df['ID_Filme'].str.replace(r'\D', '')

In [ ]:
le = LabelEncoder()

In [ ]:
df['Generos'] = le.fit_transform(df['Generos'])
df['Username'] = le.fit_transform(df['Username'])